In [2]:
import os

import tensorflow as tf
import numpy as np

print('TensorFlow version:', tf.__version__)

TensorFlow version: 1.5.0


## Представления тензоров

Способы задания матрицы

In [3]:
m1 = [[1.0, 2.0],
      [3.0, 4.0]]
m2 =  np.array([[1.0, 2.0],
                [3.0, 4.0]], dtype=np.float32)
m3 = tf.constant([[1.0, 2.0],
                  [3.0, 4.0]])

print(type(m1))
print(type(m2))
print(type(m3))

<class 'list'>
<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.Tensor'>


Приведем все объекты к одному типу явным образом

In [4]:
t1 = tf.convert_to_tensor(m1, dtype=tf.float32)
t2 = tf.convert_to_tensor(m2, dtype=tf.float32)
t3 = tf.convert_to_tensor(m3, dtype=tf.float32)
print(type(t1))
print(type(t2))
print(type(t3))

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


Пример задания объектов через tf.constant (интерес представляет подробное описание объекта при его вызове)

In [5]:
tf.reset_default_graph()

In [6]:
m1 = tf.constant([[1., 2.]]) # 2x1
m2 = tf.constant([[1.],
                  [2,]]) #1x2
m3 = tf.constant([ [[1, 2],
                    [3, 4],
                    [5, 6]],
                   [[7, 8],
                    [9, 10],
                    [11, 12]]
                 ])
print(m1)
print(m2)
print(m3)

Tensor("Const:0", shape=(1, 2), dtype=float32)
Tensor("Const_1:0", shape=(2, 1), dtype=float32)
Tensor("Const_2:0", shape=(2, 3, 2), dtype=int32)


In [7]:
tf.ones([500, 500]) * 0.5

<tf.Tensor 'mul:0' shape=(500, 500) dtype=float32>

## Создание операторов 

Оператор изменения знака

In [8]:
tf.reset_default_graph()

In [9]:
x = tf.constant([[1., 2.]])
negMatrix = tf.negative(x)
print(negMatrix)

Tensor("Neg:0", shape=(1, 2), dtype=float32)


Пример гауссовой функции

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/b77e66e0cfdc9859e63d68b5309bb794ad7cd1aa">

In [10]:
mean = 0.0
sigma = 1.0

gausDistr = (1.0/(sigma * tf.sqrt(2*np.pi)))*tf.exp(tf.negative(tf.pow((x - mean), 2))/(2 * tf.pow(sigma, 2)))

In [11]:
gausDistr

<tf.Tensor 'mul_2:0' shape=(1, 2) dtype=float32>

## Выполнене операторов во время сеанса

Явное использование tf.Sess

In [12]:
with tf.Session() as sess:
    neg_result = sess.run(negMatrix)
    distr_result = sess.run(gausDistr)
print(neg_result, distr_result)

[[-1. -2.]] [[0.24197073 0.05399097]]


Применение интерактивного режима отладки

In [13]:
sess = tf.InteractiveSession()

neg_result = negMatrix.eval()
distr_result = gausDistr.eval()
print(neg_result, distr_result)

sess.close() # Не забывать закрывать сессию!

[[-1. -2.]] [[0.24197073 0.05399097]]


Пример настройки конфигурации сеансов - вывод устрйоства, на котором будут производиться вычисления<br>
*Примечание - выводит логи в терминал, не в jupyter notebook*

In [14]:
tf.reset_default_graph()

x = tf.constant([[1., 2.]])
negMatrix = tf.negative(x)

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    result = sess.run(negMatrix)
print(result)
# (Const)/job:localhost/replica:0/task:0/device:CPU:0

[[-1. -2.]]


## Использование переменных

In [15]:
tf.reset_default_graph()

sess = tf.InteractiveSession()

raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13.]
spike = tf.Variable(False) #Create boolean variable
spike.initializer.run() # Инициализируем переменную, вызвав run на ее инициализаторе

for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i - 1] > 5:
        updater = tf.assign(spike, True) 
        updater.eval() # Обновляем значение переменной
    else:
        tf.assign(spike, False).eval() # Можно обновить значение напрямую, без updater
    print('Spike value:', spike.eval()) # Вычисляем значение переменной после обновления
sess.close()
        

Spike value: False
Spike value: True
Spike value: False
Spike value: False
Spike value: True
Spike value: False
Spike value: True


## Сохранение и загрузка переменных

In [20]:
! mkdir ./data

In [21]:
tf.reset_default_graph()

sess = tf.InteractiveSession()

raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13.]
spikes = tf.Variable([False] * len(raw_data), name = 'spikes') #Зададим булевой вектор для определение мест всплесков
spikes.initializer.run()

saver = tf.train.Saver()

for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i - 1] > 5:
        spikes_val = spikes.eval() #Вычисяем текущие значения spikes, чтобы обновить их через assign
        spikes_val[i] = True
        tf.assign(spikes, spikes_val).eval()
print(spikes.eval())

save_path = saver.save(sess, './data/chapter_2/spikes.ckpt')
print('Spikes data saved in file:', save_path)

sess.close()

[False False  True False False  True False  True]
Spikes data saved in file: ./data/chapter_2/spikes.ckpt


In [22]:
os.listdir('./data/chapter_2/')

['spikes.ckpt.meta',
 'spikes.ckpt.index',
 'spikes.ckpt.data-00000-of-00001',
 'checkpoint']

In [23]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

spikes = tf.Variable([False] * len(raw_data), name = 'spikes')
# spikes.initializer.run() - В инициализации теперь нет необходимости, так как переменная будут загружена из cpkt файла
saver = tf.train.Saver()

saver.restore(sess, './data/chapter_2/spikes.ckpt')
print(spikes.eval())

sess.close()

INFO:tensorflow:Restoring parameters from ./data/chapter_2/spikes.ckpt
[False False  True False False  True False  True]


## Визаулизация данных с помощью TensorBoard

Рассмотрим применение Tensorboard на примере скользящего среднего (экспотенциального усреднения)

udpate_avg = alpha * curr_value + (1 - alpha) * prev_avg

In [24]:
tf.reset_default_graph()

raw_data = np.random.normal(10, 1, 100)

alpha = tf.constant(0.05)
curr_value = tf.placeholder(tf.float32)
prev_avg = tf.Variable(0.) # Инициализируем предыдущее сренее нулем
update_avg = alpha * curr_value + (1 - alpha) * prev_avg

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(len(raw_data)):
        curr_avg = sess.run(update_avg, feed_dict={curr_value: raw_data[i]})
        sess.run(tf.assign(prev_avg, curr_avg))
        if i % 10 == 0:
            print(raw_data[i], curr_avg)

9.832127935430794 0.49160638
10.520354459317115 4.59555
10.296082694595235 6.8647747
8.163553024455833 7.9613667
10.446441528036011 8.782331
9.70348496649299 9.308358
12.782365252841014 9.705023
9.841376499928769 9.720145
9.359749168562317 9.845043
10.694061630144828 9.898213


Аннотирование алгоритма

In [25]:
! mkdir ./data/chapter_2/logs

In [26]:
tf.reset_default_graph()

raw_data = np.random.normal(10, 1, 100)

alpha = tf.constant(0.05)
curr_value = tf.placeholder(tf.float32)
prev_avg = tf.Variable(0.) # Инициализируем предыдущее сренее нулем
update_avg = alpha * curr_value + (1 - alpha) * prev_avg

# Добавляем в граф вычислений переменные TensorBoard
avg_hist = tf.summary.scalar('running_average', update_avg)
value_hist = tf.summary.scalar('incoming_values', curr_value)
merged = tf.summary.merge_all()

# Инициализируем директорию для записей логов
writer = tf.summary.FileWriter('./data/chapter_2/logs')

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(len(raw_data)):
        summary_str, curr_avg = sess.run([merged, update_avg], 
                                         feed_dict={curr_value: raw_data[i]})
        sess.run(tf.assign(prev_avg, curr_avg))
        writer.add_summary(summary_str, i)
        if i % 10 == 0:
            print(raw_data[i], curr_avg)

10.176407612273369 0.5088204
11.99210994861076 4.292278
11.306955398129025 6.8103933
10.425716355354286 8.182865
10.349362105538791 8.953411
10.508888612590983 9.576686
9.956042053362129 9.8736725
11.68689234525142 9.958948
9.640649180990565 10.044237
10.163183803150515 9.9187975


In [27]:
# View logs
! tensorboard --logdir=./data/chapter_2/logs

/home/grigoriy/anaconda3/envs/tf1.5/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/grigoriy/anaconda3/envs/tf1.5/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/grigoriy/anaconda3/envs/tf1.5/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/grigoriy/anaconda3/envs/tf1.5/lib/pyt

In [28]:
# # Delete logs
# ! rm -r ./data/chapter_2/logs/*